In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Creating Training and Validation data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the paths for the data
train_data_dir = 'train'
img_size = 224

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,         # Rescale pixel values to the range [0, 1]
    shear_range=0.2,        # Apply shear transformations
    zoom_range=0.2,         # Apply zoom transformations
    horizontal_flip=True,   # Apply horizontal flips
    validation_split=0.2    # Split 20% of the data for validation
)

# Training data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=64,
    class_mode='categorical',
    subset='training'       # Use a subset of the data for training
)

# Validation data
validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=64,
    class_mode='categorical',
    subset='validation'     # Use a subset of the data for validation
)


## Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Create the CNN model
model = Sequential()

# 1. Convolutional Layer
model.add(Conv2D(48, (3, 3), input_shape=(img_size, img_size, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2. Convolutional Layer
model.add(Conv2D(96, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3. Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Global Average Pooling Layer
model.add(GlobalAveragePooling2D())

# Fully connected Layer (Dense Layer)
model.add(Dense(384, activation='relu'))

# Output Layer
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

optimizer = Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)